In [2]:
#Import H2O and other libraries that will be used in this tutorial 
import h2o
import matplotlib as plt

#Import the Estimators
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

#Import h2o grid search 
import h2o.grid 
from h2o.grid.grid_search import H2OGridSearch

In [4]:
import h2o
h2o.init(max_mem_size=16)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/ubuntu/Anaconda/envs/keras_env/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpx1fep824
  JVM stdout: /tmp/tmpx1fep824/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpx1fep824/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54325
Connecting to H2O server at http://127.0.0.1:54325 ... successful.


H2O cluster uptime:,06 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,1 month and 12 days
H2O cluster name:,H2O_from_python_unknownUser_s33xun
H2O cluster total nodes:,1
H2O cluster free memory:,16 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


## Import a sample binary outcome train/test set into H2O
#### https://archive.ics.uci.edu/ml/datasets/HIGGS
#### The data has been produced using Monte Carlo simulations. The first 21 features (columns 2-22) are kinematic properties measured by the particle detectors in the accelerator. The last seven features are functions of the first 21 features; these are high-level features derived by physicists to help discriminate between the two classes. There is an interest in using deep learning methods to obviate the need for physicists to manually develop such features. Target variable is binary, 0 for background noise, 1 for signal

## AutoML
http://docs.h2o.ai/h2o/latest-stable/h2o-r/docs/reference/h2o.automl.html

#### The models are ranked by a default metric based on the problem type (the second column of the leaderboard). In binary classification problems, that metric is AUC, and in multiclass classification problems, the metric is mean per-class error. In regression problems, the default sort metric is deviance.

In [5]:
# Binary classification, AutoML finds the best algorithm and model

from h2o.automl import H2OAutoML, get_leaderboard


train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
# max_runtime_secs - maximum runtime for automl, default is set to 1 hour
# balance_classes - Set to True to over or undersample the classes to balance it
# modeling_plan - can pass a list of models to try out, the result of aml.modeling_steps can be passed back to it during initialization
# sort_metric - 'auc', 'aucpr' etc.
# export_checkpoints_dir - provide the name of the folder to checkpoint the model
# include_algos - algorithm to use, "DRF", "DeepLearning", "XGBoost" etc
# exclude_algos - exclude certain algorithms

aml = H2OAutoML(max_models=20, seed=1, project_name="First", max_runtime_secs=1000)
aml.train(x=x, y=y, training_frame=train) # provide data and response column names from the train H20 dataframe

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20200304_162954,0.788442,0.553275,0.805975,0.321207,0.432995,0.187485,23310,2.15807
StackedEnsemble_BestOfFamily_AutoML_20200304_162954,0.788303,0.553217,0.80552,0.317806,0.432977,0.187469,8801,0.677088
XGBoost_3_AutoML_20200304_162954,0.786708,0.555684,0.804735,0.317438,0.433971,0.188331,4215,0.010285
XGBoost_grid__1_AutoML_20200304_162954_model_3,0.785959,0.557668,0.803676,0.324007,0.434596,0.188874,4583,0.012497
XGBoost_grid__1_AutoML_20200304_162954_model_4,0.782356,0.559426,0.799898,0.330644,0.435863,0.189977,3124,0.010361
GBM_5_AutoML_20200304_162954,0.78219,0.558353,0.800234,0.319658,0.435512,0.18967,6381,0.260413
XGBoost_2_AutoML_20200304_162954,0.782155,0.557366,0.801728,0.337128,0.435331,0.189514,11845,0.017441
XGBoost_grid__1_AutoML_20200304_162954_model_1,0.782142,0.560727,0.799441,0.324183,0.436255,0.190319,8201,0.010943
XGBoost_1_AutoML_20200304_162954,0.7819,0.557833,0.801298,0.33284,0.435508,0.189667,8237,0.011506
GBM_2_AutoML_20200304_162954,0.777673,0.562514,0.796181,0.334056,0.437583,0.191479,4776,0.233908


In [6]:
train.describe()

Rows:10000
Cols:29




,response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
type,enum,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,,0.2750626504421234,-2.425236463546753,-1.7425082921981812,0.012355248443782328,-1.7437547445297241,0.15948815643787384,-2.9410083293914795,-1.7412374019622803,0.0,0.189988374710083,-2.910175323486328,-1.7423716783523557,0.0,0.26360762119293213,-2.727842330932617,-1.7420687675476072,0.0,0.36535415053367615,-2.496431827545166,-1.7421357631683347,0.0,0.1722409576177597,0.34246698021888733,0.46118336915969854,0.384410560131073,0.08098646253347397,0.38877949118614197,0.4449557662010193
mean,,0.9979236403375864,-0.016680663833495066,-0.0034857744504741277,0.9913848843651824,-0.009822067860461522,0.9920579565405846,-0.0014677957667277976,0.0037509448854136282,1.00493906686306,0.9877106610402465,-0.020984136478627995,-0.00016777858562709903,0.9810776063561439,0.9945834684580565,0.013843917558944673,0.007515283621835988,1.0034907880783084,0.97978388197124,0.0012221027551102396,-0.014788587855815424,0.9991417586565018,1.0291478910103438,1.021455474603176,1.0508769570112229,1.0125336340218787,0.9677125638335943,1.0312243017762897,0.9578644521445038
maxs,,6.695387840270996,2.4299979209899902,1.7432359457015991,5.824007034301758,1.7428184747695923,7.064657211303711,2.9696741104125977,1.741453766822815,2.1730761528015137,5.192512035369873,2.9093241691589355,1.7431747913360596,2.214872121810913,6.523279190063477,2.7272779941558842,1.741774320602417,2.548224449157715,6.067653179168701,2.4955105781555176,1.7428174018859863,3.101961374282837,13.098125457763672,7.3919677734375,3.6822597980499268,6.5831212997436515,8.255083084106445,4.749468803405762,4.3163652420043945
sigma,,0.5749652323165785,1.0029429816464286,1.0108375148072333,0.59546048128496,1.004827635002585,0.47740822779756953,1.0041150858680707,1.0159068205331792,1.0269649157432101,0.4899343832646299,1.009461647840542,1.0053419750816672,1.045674383534585,0.485816821742571,1.0169739644992715,0.9970667134235766,1.1953415446637299,0.4991268289853618,1.0054594863334043,1.0009981151815668,1.3969915502672692,0.6372254799960355,0.36962268165272505,0.1659393548346413,0.40492650011247683,0.523194789843795,0.36622010143683376,0.3133368626304032
zeros,,0,0,0,0,0,0,0,0,4870,0,0,0,5093,0,0,0,5688,0,0,0,6468,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0.869293212890625,-0.6350818276405334,0.22569026052951813,0.327470064163208,-0.6899932026863098,0.7542022466659546,-0.24857313930988315,-1.0920639038085938,0.0,1.3749921321868896,-0.6536741852760315,0.9303491115570068,1.1074360609054565,1.138904333114624,-1.5781983137130735,-1.046985387802124,0.0,0.6579295396804808,-0.010454569943249226,-0.0457671694457531,3.101961374282837,1.353760004043579,0.9795631170272827,0.978076159954071,0.9200048446655273,0.7216574549674988,0.9887509346008301,0.8766783475875854
1,1,0.9075421094894409,0.3291472792625427,0.3594118654727936,1.4979698657989502,-0.3130095303058624,1.09553062915802,-0.5575249195098877,-1.5882297754287722,2.1730761528015137,0.8125811815261841,-0.2136419266462326,1.2710145711898804,2.214872121810913,0.4999939501285553,-1.2614318132400513,0.7321561574935913,0.0,0.39870089292526245,-1.138930082321167,-0.0008191101951524615,0.0,0.3022198975086212,0.8330481648445129,0.9856996536254883,0.9780983924865723,0.7797321677207947,0.9923557639122009,0.7983425855636597
2,1,0.7988347411155701,1.4706387519836426,-1.6359747648239136,0.45377317070961,0.4256291687488556,1.104874610900879,1.2823222875595093,1.3816642761230469,0.0,0.8517372012138367,1.540658950805664,-0.8196895122528076,2.214872121810913,0.9934899210929871,0.3560801148414612,-0.2087775468826294,2.548224449157715,1.256954550743103,1.128847599029541,0.9004608392715454,0.0,0.9097532629966736,1.108330488204956,0.9856922030448914,0.9513312578201294,0.80325150

In [225]:
train['response']

response
1
1
1
0
1
0
1
1
1
1


### Look at the object returned by AutoML

In [90]:
help(aml)

Help on H2OAutoML in module h2o.automl.autoh2o object:

class H2OAutoML(h2o.base.Keyed)
 |  Automatic Machine Learning
 |  
 |  The Automatic Machine Learning (AutoML) function automates the supervised machine learning model training process.
 |  The current version of AutoML trains and cross-validates 
 |  a Random Forest (DRF), 
 |  an Extremely-Randomized Forest (DRF/XRT),
 |  a random grid of Generalized Linear Models (GLM)
 |  a random grid of XGBoost (XGBoost),
 |  a random grid of Gradient Boosting Machines (GBM), 
 |  a random grid of Deep Neural Nets (DeepLearning), 
 |  and 2 Stacked Ensembles, one of all the models, and one of only the best models of each kind.
 |  
 |  :examples:
 |  >>> import h2o
 |  >>> from h2o.automl import H2OAutoML
 |  >>> h2o.init()
 |  >>> # Import a sample binary outcome train/test set into H2O
 |  >>> train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
 |  >>> test = h2o.import_file("https://s3.amazonaws.com/er

In [7]:
aml.modeling_steps

[{'name': 'XGBoost',
  'steps': [{'id': 'def_1', 'weight': 10},
   {'id': 'def_2', 'weight': 10},
   {'id': 'def_3', 'weight': 10}]},
 {'name': 'GLM', 'steps': [{'id': 'def_1', 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'def_1', 'weight': 10}]},
 {'name': 'GBM',
  'steps': [{'id': 'def_1', 'weight': 10},
   {'id': 'def_2', 'weight': 10},
   {'id': 'def_3', 'weight': 10},
   {'id': 'def_4', 'weight': 10},
   {'id': 'def_5', 'weight': 10}]},
 {'name': 'DeepLearning', 'steps': [{'id': 'def_1', 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'XRT', 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'grid_1', 'weight': 100}]},
 {'name': 'GBM', 'steps': [{'id': 'grid_1', 'weight': 60}]},
 {'name': 'DeepLearning',
  'steps': [{'id': 'grid_1', 'weight': 20}, {'id': 'grid_2', 'weight': 20}]},
 {'name': 'StackedEnsemble',
  'steps': [{'id': 'best', 'weight': 10}, {'id': 'all', 'weight': 10}]}]

In [8]:
aml.training_info

{'creation_epoch': '1583339394',
 'start_epoch': '1583339394',
 'stop_epoch': '1583340335',
 'duration_secs': '940'}

### The Leaderboard contains information about all the models that were trained

In [9]:
aml.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200304_162954,0.788442,0.553275,0.805975,0.321207,0.432995,0.187485
StackedEnsemble_BestOfFamily_AutoML_20200304_162954,0.788303,0.553217,0.80552,0.317806,0.432977,0.187469
XGBoost_3_AutoML_20200304_162954,0.786708,0.555684,0.804735,0.317438,0.433971,0.188331
XGBoost_grid__1_AutoML_20200304_162954_model_3,0.785959,0.557668,0.803676,0.324007,0.434596,0.188874
XGBoost_grid__1_AutoML_20200304_162954_model_4,0.782356,0.559426,0.799898,0.330644,0.435863,0.189977
GBM_5_AutoML_20200304_162954,0.78219,0.558353,0.800234,0.319658,0.435512,0.18967
XGBoost_2_AutoML_20200304_162954,0.782155,0.557366,0.801728,0.337128,0.435331,0.189514
XGBoost_grid__1_AutoML_20200304_162954_model_1,0.782142,0.560727,0.799441,0.324183,0.436255,0.190319
XGBoost_1_AutoML_20200304_162954,0.7819,0.557833,0.801298,0.33284,0.435508,0.189667
GBM_2_AutoML_20200304_162954,0.777673,0.562514,0.796181,0.334056,0.437583,0.191479


In [10]:
# Get the leader or the best-performing model
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200304_162954

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.10485340600532099
RMSE: 0.32381075646945545
LogLoss: 0.3606686672196795
Null degrees of freedom: 9999
Residual degrees of freedom: 9984
Null deviance: 13828.113387424273
Residual deviance: 7213.37334439359
AIC: 7245.37334439359
AUC: 0.9544619821598985
AUCPR: 0.9578558742469071
Gini: 0.908923964319797

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.471149651015135: 


,,0,1,Error,Rate
0,0,3884.0,821.0,0.1745,(821.0/4705.0)
1,1,403.0,4892.0,0.0761,(403.0/5295.0)
2,Total,4287.0,5713.0,0.1224,(1224.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.471150,0.888808,220.0
1,max f2,0.352073,0.931394,265.0
2,max f0point5,0.619651,0.900916,160.0
3,max accuracy,0.502720,0.878400,207.0
4,max precision,0.939997,1.000000,0.0
5,max recall,0.174155,1.000000,347.0
6,max specificity,0.939997,1.000000,0.0
7,max absolute_mcc,0.484611,0.756501,215.0
8,max min_per_class_accuracy,0.528787,0.875452,198.0
9,max mean_per_class_accuracy,0.508695,0.877088,205.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 53.21 %


...

,,0,1,Error,Rate
0,0,2292.0,2413.0,0.5129,(2413.0/4705.0)
1,1,686.0,4609.0,0.1296,(686.0/5295.0)
2,Total,2978.0,7022.0,0.3099,(3099.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.347352,0.748397,272.0
1,max f2,0.180696,0.860286,346.0
2,max f0point5,0.644605,0.740608,141.0
3,max accuracy,0.510873,0.713300,200.0
4,max precision,0.937952,1.000000,0.0
5,max recall,0.084127,1.000000,390.0
6,max specificity,0.937952,1.000000,0.0
7,max absolute_mcc,0.604167,0.427303,159.0
8,max min_per_class_accuracy,0.528203,0.712009,193.0
9,max mean_per_class_accuracy,0.549065,0.713693,183.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.94 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.01,0.914408,1.831917,1.831917,0.970,0.922426,0.970000,0.922426,0.018319,0.018319,83.191690,83.191690
1,,2,0.02,0.905734,1.813031,1.822474,0.960,0.910038,0.965000,0.916232,0.018130,0.036449,81.303116,82.247403
2,,3,0.03,0.899971,1.831917,1.825622,0.970,0.902863,0.966667,0.911775,0.018319,0.054769,83.191690,82.562166
3,,4,0.04,0.895068,1.775260,1.813031,0.940,0.897462,0.960000,0.908197,0.017753,0.072521,77.525968,81.303116
4,,5,0.05,0.888587,1.775260,1.805477,0.940,0.891938,0.956000,0.904945,0.017753,0.090274,77.525968,80.547686
5,,6,0.10,0.861741,1.677054,1.741265,0.888,0.875300,0.922000,0.890123,0.083853,0.174127,67.705382,74.126534
6,,7,0.15,0.828992,1.643059,1.708530,0.870,0.845419,0.904667,0.875221,0.082153,0.256280,64.305949,70.853006
7,,8,0.20,0.799446,1.522191,1.661945,0.806,0.814317,0.880000,0.859995,0.076110,0.332389,52.219075,66.194523
8,,9,0.30,0.724047,1.376771,1.566887,0.729,0.763225,0.829667,0.827738,0.137677,0.470066,37.677054,56.688700
9,,10,0.40,0.637191,1.246459,1.486780,0.660,0.681038,0.787250,0.791063,0.124646,0.594712,24.645892,48.677998


In [11]:
# Get the performance of the leading model on the test set
aml.leader.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.1852630838014019
RMSE: 0.4304219834086102
LogLoss: 0.54882958718278
Null degrees of freedom: 4999
Residual degrees of freedom: 4984
Null deviance: 6905.1964032609185
Residual deviance: 5488.2958718278
AIC: 5520.2958718278
AUC: 0.7925093813723952
AUCPR: 0.8118735161426781
Gini: 0.5850187627447905

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3784157456479748: 


,,0,1,Error,Rate
0,0,1258.0,1057.0,0.4566,(1057.0/2315.0)
1,1,399.0,2286.0,0.1486,(399.0/2685.0)
2,Total,1657.0,3343.0,0.2912,(1456.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.378416,0.758461,260.0
1,max f2,0.168668,0.862520,357.0
2,max f0point5,0.638253,0.750601,144.0
3,max accuracy,0.442195,0.718800,233.0
4,max precision,0.935385,1.000000,0.0
5,max recall,0.078589,1.000000,393.0
6,max specificity,0.935385,1.000000,0.0
7,max absolute_mcc,0.583605,0.436171,169.0
8,max min_per_class_accuracy,0.515155,0.715456,200.0
9,max mean_per_class_accuracy,0.506867,0.717616,203.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 52.58 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.01,0.915017,1.787709,1.787709,0.960,0.921387,0.960000,0.921387,0.017877,0.017877,78.770950,78.770950
1,,2,0.02,0.906917,1.824953,1.806331,0.980,0.910924,0.970000,0.916155,0.018250,0.036127,82.495345,80.633147
2,,3,0.03,0.901415,1.824953,1.812539,0.980,0.904093,0.973333,0.912134,0.018250,0.054376,82.495345,81.253880
3,,4,0.04,0.895674,1.750466,1.797020,0.940,0.898633,0.965000,0.908759,0.017505,0.071881,75.046555,79.702048
4,,5,0.05,0.891197,1.675978,1.772812,0.900,0.893507,0.952000,0.905709,0.016760,0.088641,67.597765,77.281192
5,,6,0.10,0.861134,1.683426,1.728119,0.904,0.876094,0.928000,0.890901,0.084171,0.172812,68.342644,72.811918
6,,7,0.15,0.830811,1.534451,1.663563,0.824,0.846655,0.893333,0.876152,0.076723,0.249534,53.445065,66.356300
7,,8,0.20,0.796684,1.594041,1.646182,0.856,0.813792,0.884000,0.860562,0.079702,0.329236,59.404097,64.618250
8,,9,0.30,0.712316,1.407821,1.566729,0.756,0.756018,0.841333,0.825714,0.140782,0.470019,40.782123,56.672874
9,,10,0.40,0.623737,1.255121,1.488827,0.674,0.670116,0.799500,0.786815,0.125512,0.595531,25.512104,48.882682


### What if you want to select a particular model?

In [12]:
model_id_list = aml.leaderboard['model_id'].as_data_frame()

In [13]:
# Select a model using the model id
model_id = model_id_list['model_id'].loc[1]
model_id_2 = model_id_list['model_id'].loc[2] 

# Get the model using the model_id from the list
h2o.get_model(model_id)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20200304_162954

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.10508034221923618
RMSE: 0.32416098195069093
LogLoss: 0.3599137081544586
Null degrees of freedom: 9999
Residual degrees of freedom: 9994
Null deviance: 13828.113387424273
Residual deviance: 7198.274163089172
AIC: 7210.274163089172
AUC: 0.953197821617049
AUCPR: 0.9566561647684388
Gini: 0.906395643234098

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.45851575071241124: 


,,0,1,Error,Rate
0,0,3847.0,858.0,0.1824,(858.0/4705.0)
1,1,394.0,4901.0,0.0744,(394.0/5295.0)
2,Total,4241.0,5759.0,0.1252,(1252.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.458516,0.886738,222.0
1,max f2,0.333485,0.929825,272.0
2,max f0point5,0.593696,0.897210,168.0
3,max accuracy,0.458516,0.874800,222.0
4,max precision,0.942599,1.000000,0.0
5,max recall,0.181994,1.000000,338.0
6,max specificity,0.942599,1.000000,0.0
7,max absolute_mcc,0.458516,0.750635,222.0
8,max min_per_class_accuracy,0.521097,0.873114,195.0
9,max mean_per_class_accuracy,0.516717,0.874328,197.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.85 %


...

,,0,1,Error,Rate
0,0,2348.0,2357.0,0.501,(2357.0/4705.0)
1,1,713.0,4582.0,0.1347,(713.0/5295.0)
2,Total,3061.0,6939.0,0.307,(3070.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.359578,0.749060,269.0
1,max f2,0.171367,0.860520,354.0
2,max f0point5,0.607184,0.740445,159.0
3,max accuracy,0.553324,0.712000,181.0
4,max precision,0.939481,1.000000,0.0
5,max recall,0.079288,1.000000,392.0
6,max specificity,0.939481,1.000000,0.0
7,max absolute_mcc,0.592285,0.428993,165.0
8,max min_per_class_accuracy,0.528796,0.709537,192.0
9,max mean_per_class_accuracy,0.592285,0.713665,165.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.95 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.01,0.915607,1.869688,1.869688,0.990,0.923793,0.990000,0.923793,0.018697,0.018697,86.968839,86.968839
1,,2,0.02,0.906941,1.775260,1.822474,0.940,0.910720,0.965000,0.917256,0.017753,0.036449,77.525968,82.247403
2,,3,0.03,0.900423,1.813031,1.819326,0.960,0.903169,0.963333,0.912560,0.018130,0.054580,81.303116,81.932641
3,,4,0.04,0.894621,1.831917,1.822474,0.970,0.897775,0.965000,0.908864,0.018319,0.072899,83.191690,82.247403
4,,5,0.05,0.888940,1.718602,1.801700,0.910,0.891939,0.954000,0.905479,0.017186,0.090085,71.860246,80.169972
5,,6,0.10,0.859352,1.695940,1.748820,0.898,0.874857,0.926000,0.890168,0.084797,0.174882,69.593957,74.881964
6,,7,0.15,0.826923,1.597734,1.698458,0.846,0.843190,0.899333,0.874509,0.079887,0.254769,59.773371,69.845766
7,,8,0.20,0.796632,1.586402,1.670444,0.840,0.811976,0.884500,0.858876,0.079320,0.334089,58.640227,67.044381
8,,9,0.30,0.720580,1.365439,1.568776,0.723,0.759479,0.830667,0.825743,0.136544,0.470633,36.543909,56.877557
9,,10,0.40,0.635201,1.229462,1.483947,0.651,0.679844,0.785750,0.789268,0.122946,0.593579,22.946176,48.394712


In [14]:
model = h2o.get_model(model_id)
model2 = h2o.get_model(model_id_2)
help(model)

Help on H2OStackedEnsembleEstimator in module h2o.estimators.stackedensemble object:

ModelBase = class H2OStackedEnsembleEstimator(h2o.estimators.estimator_base.H2OEstimator)
 |  Stacked Ensemble
 |  
 |  Builds a stacked ensemble (aka "super learner") machine learning method that uses two
 |  or more H2O learning algorithms to improve predictive performance. It is a loss-based
 |  supervised learning method that finds the optimal combination of a collection of prediction
 |  algorithms.This method supports regression and binary classification.
 |  
 |  :examples:
 |  
 |  >>> import h2o
 |  >>> h2o.init()
 |  >>> from h2o.estimators.random_forest import H2ORandomForestEstimator
 |  >>> from h2o.estimators.gbm import H2OGradientBoostingEstimator
 |  >>> from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
 |  >>> col_types = ["numeric", "numeric", "numeric", "enum",
 |  ...              "enum", "numeric", "numeric", "numeric", "numeric"]
 |  >>> data = h2o.import_fil

In [30]:
import pprint

pprint.pprint(model.actual_params)
pprint.pprint(model.full_parameters)

{'base_models': [{'URL': '/3/Models/XGBoost_3_AutoML_20200304_162954',
                  '__meta': {'schema_name': 'ModelKeyV3',
                             'schema_type': 'Key<Model>',
                             'schema_version': 3},
                  'name': 'XGBoost_3_AutoML_20200304_162954',
                  'type': 'Key<Model>'},
                 {'URL': '/3/Models/GBM_5_AutoML_20200304_162954',
                  '__meta': {'schema_name': 'ModelKeyV3',
                             'schema_type': 'Key<Model>',
                             'schema_version': 3},
                  'name': 'GBM_5_AutoML_20200304_162954',
                  'type': 'Key<Model>'},
                 {'URL': '/3/Models/DRF_1_AutoML_20200304_162954',
                  '__meta': {'schema_name': 'ModelKeyV3',
                             'schema_type': 'Key<Model>',
                             'schema_version': 3},
                  'name': 'DRF_1_AutoML_20200304_162954',
                  'type': 'Key<Mod

### Save model

In [ ]:
# For MOJO only these are supported
# GBM (Gradient Boosting Machines)
# DRF (Distributed Random Forest)
# IRF (Isolation Random Forest)
# GLM (Generalized Linear Model)
# XGBoost

model2.download_mojo() # Won't work for StackedEnsemble

#model.download_model() # Not sure how to load this back in

model.predict(test_data=test)

In [16]:
h2o.save_model(model, 'saved')

'/home/ubuntu/keras/saved/StackedEnsemble_BestOfFamily_AutoML_20200304_162954'

In [17]:
model_loaded = h2o.load_model('/home/ubuntu/keras/saved/StackedEnsemble_AllModels_AutoML_20200227_204508')
model_loaded.predict(test_data=test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
0,0.758287,0.241713
0,0.692455,0.307545
1,0.579463,0.420537
1,0.289396,0.710604
0,0.696569,0.303431
1,0.262359,0.737641
1,0.289035,0.710965
1,0.609009,0.390991
1,0.606744,0.393256
0,0.832291,0.167709


In [18]:
mojo_loaded = h2o.import_mojo('XGBoost_3_AutoML_20200304_162954.zip')
mojo_loaded.predict(test_data=test)

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1583339346847_3082


Model Summary: 


,,number_of_trees
0,,113.0




ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.15625625666093138
RMSE: 0.39529262156145967
LogLoss: 0.48291363480005595
Mean Per-Class Error: 0.21430088136804226
AUC: 0.870783778332375
AUCPR: 0.8796833496562693
Gini: 0.7415675566647499

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.43120504220326744: 


,,0,1,Error,Rate
0,0,3067.0,1638.0,0.3481,(1638.0/4705.0)
1,1,581.0,4714.0,0.1097,(581.0/5295.0)
2,Total,3648.0,6352.0,0.2219,(2219.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-03-04 16:32:21,31.776 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.4705
1,,2020-03-04 16:32:21,32.107 sec,5.0,0.482614,0.658839,0.786043,0.795905,1.780213,0.3069
2,,2020-03-04 16:32:21,32.234 sec,10.0,0.469730,0.633733,0.798669,0.809335,1.767290,0.2921
3,,2020-03-04 16:32:21,32.333 sec,15.0,0.458053,0.610914,0.806382,0.815637,1.813031,0.2750
4,,2020-03-04 16:32:22,32.458 sec,20.0,0.449562,0.594067,0.811875,0.820995,1.869875,0.2741
5,,2020-03-04 16:32:22,32.617 sec,25.0,0.442306,0.579416,0.815830,0.825379,1.869688,0.2765
6,,2020-03-04 16:32:22,32.749 sec,30.0,0.436562,0.567691,0.821003,0.829769,1.888574,0.2729
7,,2020-03-04 16:32:22,32.893 sec,35.0,0.432161,0.558675,0.825320,0.835295,1.888574,0.2619
8,,2020-03-04 16:32:22,33.054 sec,40.0,0.428006,0.550050,0.829387,0.839608,1.888574,0.2597
9,,2020-03-04 16:32:22,33.182 sec,45.0,0.424485,0.542691,0.833368,0.843841,1.888574,0.2538



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,x26,8278.851562,1.000000,0.217662
1,x28,4533.871582,0.547645,0.119201
2,x27,3890.611328,0.469946,0.102289
3,x25,3153.177002,0.380871,0.082901
4,x6,3151.825195,0.380708,0.082866
5,x23,2836.803467,0.342657,0.074583
6,x1,1661.282593,0.200666,0.043677
7,x4,1467.694580,0.177282,0.038588
8,x10,1120.439209,0.135338,0.029458
9,x22,1103.762451,0.133323,0.029019



See the whole table with table.as_data_frame()

generic prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.746806,0.253194
0,0.63423,0.36577
0,0.631137,0.368863
1,0.359113,0.640887
0,0.676197,0.323803
1,0.314696,0.685304
1,0.337244,0.662756
0,0.560475,0.439525
0,0.602084,0.397916
0,0.671376,0.328624


### Modify the parameters of the AutoML object

In [19]:
# Binary classification, AutoML finds the best algorithm and model

from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
# max_runtime_secs - maximum runtime for automl
# balance_classes - Set to True to over or undersample the classes to balance it
# modeling_plan - can pass a list of models to try out, the result of aml.modeling_plan can be passed back to it during initialization
# sort_metric - 'auc', 'aucpr' etc.
# export_checkpoints_dir - provide the name of the folder to checkpoint the model
# include_algos - algorithm to use, "DRF", "DeepLearning", "XGBoost" etc
# exclude_algos - exclude certain algorithms

# Pass the modeling plan from the last execution, set maximum runtime
aml = H2OAutoML(max_models=20, seed=1, 
                max_runtime_secs=100,
                balance_classes=True, # Usually a good idea to set this to True
                modeling_plan=aml.modeling_steps,
                export_checkpoints_dir='/home/ubuntu/H2Omodels' # The models created get saved to this folder
               )
aml.train(x=x, y=y, training_frame=train) # provide data and response column names from the train H20 dataframe

# AutoML Leaderboard, H20 dataframe
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20200304_170927,0.785534,0.557006,0.804389,0.326616,0.434692,0.188957,1202,0.035674
XGBoost_2_AutoML_20200304_170927,0.782521,0.557374,0.801723,0.322245,0.435219,0.189415,9827,0.017385
XGBoost_1_AutoML_20200304_170927,0.7819,0.557833,0.801298,0.33284,0.435508,0.189667,6945,0.01722


In [20]:
# Sort the leaderboard by a certain column
lb.sort(by='training_time_ms').head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20200304_170927,0.785534,0.557006,0.804389,0.326616,0.434692,0.188957,1202,0.035674
XGBoost_1_AutoML_20200304_170927,0.7819,0.557833,0.801298,0.33284,0.435508,0.189667,6945,0.01722
XGBoost_2_AutoML_20200304_170927,0.782521,0.557374,0.801723,0.322245,0.435219,0.189415,9827,0.017385


### Look at the modeling plan passed to AutoML and what was used

In [21]:
# Modeling plan passed to it from the previous execution
aml.modeling_plan

[{'name': 'XGBoost',
  'steps': [{'id': 'def_1', 'weight': 10},
   {'id': 'def_2', 'weight': 10},
   {'id': 'def_3', 'weight': 10}]},
 {'name': 'GLM', 'steps': [{'id': 'def_1', 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'def_1', 'weight': 10}]},
 {'name': 'GBM',
  'steps': [{'id': 'def_1', 'weight': 10},
   {'id': 'def_2', 'weight': 10},
   {'id': 'def_3', 'weight': 10},
   {'id': 'def_4', 'weight': 10},
   {'id': 'def_5', 'weight': 10}]},
 {'name': 'DeepLearning', 'steps': [{'id': 'def_1', 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'XRT', 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'grid_1', 'weight': 100}]},
 {'name': 'GBM', 'steps': [{'id': 'grid_1', 'weight': 60}]},
 {'name': 'DeepLearning',
  'steps': [{'id': 'grid_1', 'weight': 20}, {'id': 'grid_2', 'weight': 20}]},
 {'name': 'StackedEnsemble',
  'steps': [{'id': 'best', 'weight': 10}, {'id': 'all', 'weight': 10}]}]

In [22]:
# The mpodeling steps that were actually executed
aml.modeling_steps

[{'name': 'XGBoost',
  'steps': [{'id': 'def_1', 'weight': 10},
   {'id': 'def_2', 'weight': 10},
   {'id': 'def_3', 'weight': 10}]},
 {'name': 'StackedEnsemble', 'steps': [{'id': 'all', 'weight': 10}]}]

In [23]:
lb.summary()

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
type,string,real,real,real,real,real,real,int,real
mins,NaN,0.7819000139485549,0.5570060199375269,0.801298480411238,0.32224493461740317,0.43469209488504906,0.1889572173555525,1202.0,0.01722
mean,NaN,0.7833185184293191,0.5574040874980668,0.8024704211541814,0.32723373396125244,0.4351395975234049,0.18934658338054183,5991.333333333333,0.02342633333333333
maxs,NaN,0.785534044007189,0.557832683132384,0.8043893978309902,0.33284001609603026,0.435507839700512,0.18966707844060685,9827.0,0.035674
sigma,NaN,0.0019437015173857308,0.0004141762691164522,0.00167540730534287,0.005324462567480999,0.00041360798653639275,0.0003599070676936614,4390.87307187686,0.01060711130955706
zeros,0,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0
0,StackedEnsemble_AllModels_AutoML_20200304_170927,0.785534044007189,0.5570060199375269,0.8043893978309902,0.3266162511703239,0.43469209488504906,0.1889572173555525,1202.0,0.035674
1,XGBoost_2_AutoML_20200304_170927,0.7825214973322134,0.5573735594242893,0.8017233852203163,0.32224493461740317,0.4352188579846537,0.18941545434546617,9827.0,0.017385
2,XGBoost_1_AutoML_20200304_170927,0.7819000139485549,0.557832683132384,0.801298480411238,0.33284001609603026,0.435507839700512,0.18966707844060685,6945.0,0.01722


### Continue the AutoML run on top of the first one (22 models). The new one now has 24 models

In [24]:
# Run AutoML on top of the first run

from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
# max_runtime_secs - maximum runtime for automl
# balance_classes - Set to True to over or undersample the classes to balance it
# modeling_plan - can pass a list of models to try out, the result of aml.modeling_plan can be passed back to it during initialization
# sort_metric - 'auc', 'aucpr' etc.
# export_checkpoints_dir - provide the name of the folder to checkpoint the model
# include_algos - algorithm to use, "DRF", "DeepLearning", "XGBoost" etc
# exclude_algos - exclude certain algorithms

# Pass the modeling plan from the last execution, set maximum runtime
aml = H2OAutoML(max_models=20, seed=1, 
                project_name="First", # Setting the project to First so it can start from the first run
                max_runtime_secs=100,
                balance_classes=True, # Usually a good idea to set this to True
                modeling_plan=aml.modeling_steps,
                export_checkpoints_dir='/home/ubuntu/H2Omodels' # The models created get saved to this folder
               )
aml.train(x=x, y=y, training_frame=train) # provide data and response column names from the train H20 dataframe

# AutoML Leaderboard, H20 dataframe
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
17:11:54.39: New models will be added to existing leaderboard First@@response (leaderboard frame=null) with already 22 models.

████████████████████████████████████████████████████████| 100%

17:13:36.190: StackedEnsemble_AllModels_AutoML_20200304_171154 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.



model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20200304_162954,0.788442,0.553275,0.805975,0.321207,0.432995,0.187485,23310,2.15807
StackedEnsemble_BestOfFamily_AutoML_20200304_162954,0.788303,0.553217,0.80552,0.317806,0.432977,0.187469,8801,0.677088
XGBoost_3_AutoML_20200304_162954,0.786708,0.555684,0.804735,0.317438,0.433971,0.188331,4215,0.010285
XGBoost_grid__1_AutoML_20200304_162954_model_3,0.785959,0.557668,0.803676,0.324007,0.434596,0.188874,4583,0.012497
XGBoost_2_AutoML_20200304_171154,0.782692,0.557074,0.802186,0.321571,0.435125,0.189333,9358,0.01917
XGBoost_grid__1_AutoML_20200304_162954_model_4,0.782356,0.559426,0.799898,0.330644,0.435863,0.189977,3124,0.010361
GBM_5_AutoML_20200304_162954,0.78219,0.558353,0.800234,0.319658,0.435512,0.18967,6381,0.260413
XGBoost_2_AutoML_20200304_162954,0.782155,0.557366,0.801728,0.337128,0.435331,0.189514,11845,0.017441
XGBoost_grid__1_AutoML_20200304_162954_model_1,0.782142,0.560727,0.799441,0.324183,0.436255,0.190319,8201,0.010943
XGBoost_1_AutoML_20200304_162954,0.7819,0.557833,0.801298,0.33284,0.435508,0.189667,8237,0.011506


### Metalearners are ensembles trained using strong and diverse learners, also called superlearning or stacking
### Different from bagging and boosting because you use strong learners in metalearning instead of weak learners in bagging or boosting

In [25]:
metalearner = model.metalearner()
metalearner

{'__meta': {'schema_version': 3,
  'schema_name': 'ModelKeyV3',
  'schema_type': 'Key<Model>'},
 'name': 'metalearner_AUTO_StackedEnsemble_BestOfFamily_AutoML_20200304_162954',
 'type': 'Key<Model>',
 'URL': '/3/Models/metalearner_AUTO_StackedEnsemble_BestOfFamily_AutoML_20200304_162954'}

In [26]:
meta_model = h2o.get_model(metalearner['name'])
meta_model.coef()

{'Intercept': -3.1323330254925628,
 'XGBoost_3_AutoML_20200304_162954': 3.1149546575348084,
 'GBM_5_AutoML_20200304_162954': 1.2645388546322769,
 'DRF_1_AutoML_20200304_162954': 0.8595511253479585,
 'XRT_1_AutoML_20200304_162954': 0.3129910827687832,
 'DeepLearning_1_AutoML_20200304_162954': 0.0,
 'GLM_1_AutoML_20200304_162954': 0.64075359764011}

### Train using only family of algorithms, here that is Deep Learning

In [27]:
# Run only the DL algorithm
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
# max_runtime_secs - maximum runtime for automl
# balance_classes - Set to True to over or undersample the classes to balance it
# modeling_plan - can pass a list of models to try out, the result of aml.modeling_plan can be passed back to it during initialization
# sort_metric - 'auc', 'aucpr' etc.
# export_checkpoints_dir - provide the name of the folder to checkpoint the model
# include_algos - algorithm to use, "DRF", "DeepLearning", "XGBoost" etc
# exclude_algos - exclude certain algorithms

# Pass the modeling plan from the last execution, set maximum runtime
aml = H2OAutoML(max_models=20, seed=1, 
                max_runtime_secs=500,
                balance_classes=True, # Usually a good idea to set this to True
                include_algos=['DeepLearning'],
                export_checkpoints_dir='/home/ubuntu/H2Omodels' # The models created get saved to this folder
               )
aml.train(x=x, y=y, training_frame=train) # provide data and response column names from the train H20 dataframe

# AutoML Leaderboard, H20 dataframe
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
DeepLearning_1_AutoML_20200304_171517,0.703841,0.63045,0.707663,0.405997,0.468065,0.219085,2102,0.014632
DeepLearning_grid__1_AutoML_20200304_171517_model_1,0.656239,0.693249,0.658053,0.430346,0.490793,0.240878,51003,0.038205
DeepLearning_grid__2_AutoML_20200304_171517_model_1,0.614483,1.10269,0.60043,0.461879,0.560458,0.314113,36902,0.353038


In [28]:
aml.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_20200304_171517


Status of Neuron Layers: predicting response, 2-class classification, bernoulli distribution, CrossEntropy loss, 532 weights/biases, 15.7 KB, 121,095 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,28,Input,0,,,,,,,,,
1,,2,10,Rectifier,0,0,0,0.00192839,0.000775095,0,-0.00921206,0.220964,0.302168,0.124223
2,,3,10,Rectifier,0,0,0,0.00141845,0.000778404,0,-0.0538247,0.276302,0.807658,0.10824
3,,4,10,Rectifier,0,0,0,0.0025505,0.00309433,0,-0.05104,0.372825,0.950196,0.0638258
4,,5,2,Softmax,,0,0,0.00192022,0.00071647,0,-0.266214,1.75365,0.0243743,0.00393424




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.20343415042934823
RMSE: 0.451036750641617
LogLoss: 0.5922353887471175
Mean Per-Class Error: 0.2990703217399322
AUC: 0.7607787756387449
AUCPR: 0.7466701671359594
Gini: 0.5215575512774897

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5013919312095112: 


,,0,1,Error,Rate
0,0,2832.0,2206.0,0.4379,(2206.0/5038.0)
1,1,924.0,4088.0,0.1844,(924.0/5012.0)
2,Total,3756.0,6294.0,0.3114,(3130.0/10050.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-03-04 17:15:27,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-04 17:15:27,10.329 sec,65266 obs/sec,1.038313,1,11030.0,0.484209,0.660622,0.062161,0.639564,0.62041,1.707387,0.439204
2,,2020-03-04 17:15:29,12.021 sec,69118 obs/sec,11.399322,11,121095.0,0.451037,0.592235,0.186258,0.760779,0.74667,1.925774,0.311443



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,x28,1.000000,1.000000,0.093673
1,x26,0.757800,0.757800,0.070986
2,x6,0.710810,0.710810,0.066584
3,x27,0.645156,0.645156,0.060434
4,x1,0.602057,0.602057,0.056397
5,x23,0.583545,0.583545,0.054663
6,x7,0.455931,0.455931,0.042709
7,x4,0.420640,0.420640,0.039403
8,x25,0.414296,0.414296,0.038809
9,x17,0.396930,0.396930,0.037182



See the whole table with table.as_data_frame()


### Use checkpoints to retrain a previously loaded model. 

### All the parameters must match 

In [268]:
from h2o.estimators import H2ODeepLearningEstimator

In [329]:
dl = H2ODeepLearningEstimator(model_id="New_DL_model",
                              hidden=[10,10],
                              epochs=10,                       # note epoch is provided here instead of in train
                              export_weights_and_biases=True,
                              max_runtime_secs=100)

dl.train(x=x, y=y, training_frame=train)
dl.model_performance(test)

deeplearning Model Build progress: |██████████████████████████████████████| 100%

ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.20925372324242225
RMSE: 0.4574425901054932
LogLoss: 0.6068452804178831
Mean Per-Class Error: 0.3276176341646859
AUC: 0.7309248645583214
AUCPR: 0.7377184584441348
Gini: 0.4618497291166428

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.31530399989867675: 


,,0,1,Error,Rate
0,0,752.0,1563.0,0.6752,(1563.0/2315.0)
1,1,222.0,2463.0,0.0827,(222.0/2685.0)
2,Total,974.0,4026.0,0.357,(1785.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.315304,0.734019,293.0
1,max f2,0.128287,0.858458,362.0
2,max f0point5,0.598014,0.702072,155.0
3,max accuracy,0.512027,0.674200,202.0
4,max precision,0.976612,1.000000,0.0
5,max recall,0.035362,1.000000,394.0
6,max specificity,0.976612,1.000000,0.0
7,max absolute_mcc,0.598014,0.348492,155.0
8,max min_per_class_accuracy,0.536652,0.670764,188.0
9,max mean_per_class_accuracy,0.598014,0.672382,155.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 52.01 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.01,0.917423,1.713222,1.713222,0.920,0.940780,0.920000,0.940780,0.017132,0.017132,71.322160,71.322160
1,,2,0.02,0.890555,1.601490,1.657356,0.860,0.904525,0.890000,0.922652,0.016015,0.033147,60.148976,65.735568
2,,3,0.03,0.870702,1.564246,1.626319,0.840,0.880495,0.873333,0.908600,0.015642,0.048790,56.424581,62.631906
3,,4,0.04,0.852904,1.415270,1.573557,0.760,0.862363,0.845000,0.897041,0.014153,0.062942,41.527002,57.355680
4,,5,0.05,0.836531,1.415270,1.541899,0.760,0.845146,0.828000,0.886662,0.014153,0.077095,41.527002,54.189944
5,,6,0.10,0.791037,1.452514,1.497207,0.780,0.809923,0.804000,0.848292,0.072626,0.149721,45.251397,49.720670
6,,7,0.15,0.749697,1.467412,1.487275,0.788,0.769674,0.798667,0.822086,0.073371,0.223091,46.741155,48.727498
7,,8,0.20,0.720476,1.459963,1.480447,0.784,0.735335,0.795000,0.800398,0.072998,0.296089,45.996276,48.044693
8,,9,0.30,0.663899,1.314711,1.425202,0.706,0.692488,0.765333,0.764428,0.131471,0.427561,31.471136,42.520174
9,,10,0.40,0.605688,1.303538,1.394786,0.700,0.634549,0.749000,0.731959,0.130354,0.557914,30.353818,39.478585


In [330]:
dl.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-03-04 00:50:02,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-04 00:50:02,0.411 sec,56818 obs/sec,1.0,1,10000.0,0.494697,0.686512,0.017682,0.633081,0.639882,1.491974,0.4345
2,,2020-03-04 00:50:03,1.789 sec,67430 obs/sec,10.0,10,100000.0,0.449559,0.588554,0.188764,0.750474,0.758100,1.813031,0.3543


In [307]:
dl.weights(1) # Weights of the trained model

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
-0.136051,0.244544,0.498893,0.628877,-0.33611,0.145164,0.156553,-0.0430675,0.25279,0.344514
0.568515,0.433277,0.484943,-0.0440656,0.247107,0.0688317,0.0593354,-0.203126,0.393687,-0.501645
-0.521144,-0.302357,-0.39151,-0.200167,-0.0816884,-0.0746527,-0.0793938,-0.229765,-0.453901,-0.0558551
-0.0594717,-0.23894,-0.62836,-0.34112,-0.284174,0.120231,-0.547088,-0.221218,-0.523819,0.162292
-0.414124,-0.0597832,0.308511,-0.000354173,0.188302,-0.0288818,0.197779,0.382259,-0.0705881,0.0318458
-0.287331,-0.323727,0.176089,-0.313284,0.028234,-0.0592566,0.119208,-0.0116629,-0.355036,-0.0648204
0.159053,-0.248458,0.258571,-0.282308,0.452759,-0.0641324,0.467498,0.492435,-0.166129,-0.231782
0.373426,0.138219,-0.256499,-0.0719556,0.423208,0.0086089,0.0867236,0.101754,-0.371006,-0.305707
0.0202741,-0.295892,-0.171638,0.222765,0.0472744,-0.0999544,0.0605103,0.356936,-0.528496,0.207837
-0.0641037,0.00115702,-0.393953,-0.234585,0.47508,-0.430143,-0.0999465,-0.287195,-0.139764,-0.263403


In [332]:
# Reload the model from above

# Provide the model id from the above model to the checkpoint argument of the new model, all paramaters must match exactly
dl_ch = H2ODeepLearningEstimator(model_id="New_DL_model_contd",
                              checkpoint = "New_DL_model",
                              hidden=[10,10],
                              epochs=20,                       # note epoch is provided here instead of in train
                              export_weights_and_biases=True,
                              max_runtime_secs=100)

dl_ch.train(x=x, y=y, training_frame=train)
dl_ch.model_performance(test)

deeplearning Model Build progress: |██████████████████████████████████████| 100%

ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.20788345580248038
RMSE: 0.45594238210817867
LogLoss: 0.6046006294856857
Mean Per-Class Error: 0.3220909701525554
AUC: 0.7369904798677558
AUCPR: 0.7467691891266097
Gini: 0.4739809597355116

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34415432733651147: 


,,0,1,Error,Rate
0,0,771.0,1544.0,0.667,(1544.0/2315.0)
1,1,220.0,2465.0,0.0819,(220.0/2685.0)
2,Total,991.0,4009.0,0.3528,(1764.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.344154,0.736480,292.0
1,max f2,0.140649,0.857990,368.0
2,max f0point5,0.568702,0.702546,189.0
3,max accuracy,0.547129,0.678800,199.0
4,max precision,0.993548,1.000000,0.0
5,max recall,0.039672,1.000000,396.0
6,max specificity,0.993548,1.000000,0.0
7,max absolute_mcc,0.568702,0.355121,189.0
8,max min_per_class_accuracy,0.572075,0.676890,187.0
9,max mean_per_class_accuracy,0.568702,0.677909,189.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 56.35 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.01,0.952764,1.713222,1.713222,0.920,0.965184,0.920000,0.965184,0.017132,0.017132,71.322160,71.322160
1,,2,0.02,0.932582,1.638734,1.675978,0.880,0.941580,0.900000,0.953382,0.016387,0.033520,63.873371,67.597765
2,,3,0.03,0.919043,1.489758,1.613904,0.800,0.925559,0.866667,0.944108,0.014898,0.048417,48.975791,61.390441
3,,4,0.04,0.906147,1.713222,1.638734,0.920,0.912663,0.880000,0.936247,0.017132,0.065549,71.322160,63.873371
4,,5,0.05,0.897141,1.564246,1.623836,0.840,0.901486,0.872000,0.929294,0.015642,0.081192,56.424581,62.383613
5,,6,0.10,0.855501,1.437616,1.530726,0.772,0.875940,0.822000,0.902617,0.071881,0.153073,43.761639,53.072626
6,,7,0.15,0.820416,1.527002,1.529485,0.820,0.836712,0.821333,0.880649,0.076350,0.229423,52.700186,52.948479
7,,8,0.20,0.785196,1.415270,1.500931,0.760,0.802582,0.806000,0.861132,0.070764,0.300186,41.527002,50.093110
8,,9,0.30,0.725948,1.333333,1.445065,0.716,0.755024,0.776000,0.825763,0.133333,0.433520,33.333333,44.506518
9,,10,0.40,0.658702,1.199255,1.383613,0.644,0.693058,0.743000,0.792587,0.119926,0.553445,19.925512,38.361266


In [333]:
dl_ch.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-03-04 00:50:02,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-04 00:50:02,0.411 sec,56818 obs/sec,1.0,1,10000.0,0.494697,0.686512,0.017682,0.633081,0.639882,1.491974,0.4345
2,,2020-03-04 00:50:03,1.789 sec,67430 obs/sec,10.0,10,100000.0,0.449559,0.588554,0.188764,0.750474,0.758100,1.813031,0.3543
3,,2020-03-04 00:51:20,2.270 sec,68259 obs/sec,12.0,11,120000.0,0.449795,0.589443,0.187910,0.749887,0.755192,1.794145,0.3464
4,,2020-03-04 00:51:21,3.319 sec,74294 obs/sec,20.0,15,200000.0,0.447259,0.584562,0.197041,0.758914,0.764518,1.775260,0.3333


In [370]:
train = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/smalldata/iris/iris_wheader.csv")

splits = train.split_frame(ratios=[0.75], seed=1234)

x = splits[0].columns
y = "class"
x.remove(y)

dl = H2ODeepLearningEstimator(model_id="New_iris",
                              hidden=[4,4],
                              epochs=20,                       # note epoch is provided here instead of in train
                              export_weights_and_biases=True,
                              max_runtime_secs=100)

dl.train(x=x, y=y, training_frame=splits[0])

dl.model_performance(splits[1])

Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%

ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.053887697807961245
RMSE: 0.23213723916675078
LogLoss: 0.22072045988785519
Mean Per-Class Error: 0.05555555555555555

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,15.0,0.0,0.0,0.000000,0 / 15
1,0.0,10.0,2.0,0.166667,2 / 12
2,0.0,0.0,6.0,0.000000,0 / 6
3,15.0,10.0,8.0,0.060606,2 / 33



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.939394
1,2,1.000000
2,3,1.000000


In [375]:
# Look at the network, variable importance is turned off for large networks (default)
dl

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  New_iris_contd


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 55 weights/biases, 4.1 KB, 3,202 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,4,Input,0,,,,,,,,,
1,,2,4,Rectifier,0,0,0,0.00138081,0.000785649,0,0.0502182,0.643404,0.564398,0.322376
2,,3,4,Rectifier,0,0,0,0.000623296,0.000259198,0,-0.0715868,0.549129,0.813443,0.41253
3,,4,3,Softmax,,0,0,0.015087,0.027768,0,-0.597676,1.98736,0.916507,1.65428




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.03612705376175635
RMSE: 0.19007118077645635
LogLoss: 0.1597239412917021
Mean Per-Class Error: 0.033891547049441785

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,35.0,0.0,0.0,0.000000,0 / 35
1,0.0,35.0,3.0,0.078947,3 / 38
2,0.0,1.0,43.0,0.022727,1 / 44
3,35.0,36.0,46.0,0.034188,4 / 117



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.965812
1,2,1.000000
2,3,1.000000



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
0,,2020-03-04 01:10:36,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN
1,,2020-03-04 01:10:36,0.046 sec,78000 obs/sec,2.000000,1,234.0,0.679082,1.384462,0.310990,0.623932
2,,2020-03-04 01:10:36,0.071 sec,90000 obs/sec,20.000000,10,2340.0,0.233788,0.213118,0.918337,0.051282
3,,2020-03-04 01:11:08,0.201 sec,57239 obs/sec,22.504274,11,2633.0,0.212239,0.196136,0.932698,0.051282
4,,2020-03-04 01:11:08,0.213 sec,57178 obs/sec,27.367521,13,3202.0,0.190071,0.159724,0.946022,0.034188



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,petal_wid,1.000000,1.000000,0.339784
1,petal_len,0.745005,0.745005,0.253141
2,sepal_len,0.627780,0.627780,0.213310
3,sepal_wid,0.570262,0.570262,0.193766


In [373]:
dl = H2ODeepLearningEstimator(model_id="New_iris_contd",
                              checkpoint="New_iris",
                              hidden=[4,4],
                              epochs=25,                       # note epoch is provided here instead of in train
                              export_weights_and_biases=True,
                              max_runtime_secs=100)

dl.train(x=x, y=y, training_frame=splits[0])

dl.model_performance(splits[1])

deeplearning Model Build progress: |██████████████████████████████████████| 100%

ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.032482450301030606
RMSE: 0.1802288830932229
LogLoss: 0.150628045856322
Mean Per-Class Error: 0.027777777777777776

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,15.0,0.0,0.0,0.000000,0 / 15
1,0.0,11.0,1.0,0.083333,1 / 12
2,0.0,0.0,6.0,0.000000,0 / 6
3,15.0,11.0,7.0,0.030303,1 / 33



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.969697
1,2,1.000000
2,3,1.000000


In [374]:
# Look at the network, variable importance is turned off for large networks (default)
dl 

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  New_iris_contd


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 55 weights/biases, 4.1 KB, 3,202 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,4,Input,0,,,,,,,,,
1,,2,4,Rectifier,0,0,0,0.00138081,0.000785649,0,0.0502182,0.643404,0.564398,0.322376
2,,3,4,Rectifier,0,0,0,0.000623296,0.000259198,0,-0.0715868,0.549129,0.813443,0.41253
3,,4,3,Softmax,,0,0,0.015087,0.027768,0,-0.597676,1.98736,0.916507,1.65428




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.03612705376175635
RMSE: 0.19007118077645635
LogLoss: 0.1597239412917021
Mean Per-Class Error: 0.033891547049441785

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,35.0,0.0,0.0,0.000000,0 / 35
1,0.0,35.0,3.0,0.078947,3 / 38
2,0.0,1.0,43.0,0.022727,1 / 44
3,35.0,36.0,46.0,0.034188,4 / 117



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.965812
1,2,1.000000
2,3,1.000000



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
0,,2020-03-04 01:10:36,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN
1,,2020-03-04 01:10:36,0.046 sec,78000 obs/sec,2.000000,1,234.0,0.679082,1.384462,0.310990,0.623932
2,,2020-03-04 01:10:36,0.071 sec,90000 obs/sec,20.000000,10,2340.0,0.233788,0.213118,0.918337,0.051282
3,,2020-03-04 01:11:08,0.201 sec,57239 obs/sec,22.504274,11,2633.0,0.212239,0.196136,0.932698,0.051282
4,,2020-03-04 01:11:08,0.213 sec,57178 obs/sec,27.367521,13,3202.0,0.190071,0.159724,0.946022,0.034188



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,petal_wid,1.000000,1.000000,0.339784
1,petal_len,0.745005,0.745005,0.253141
2,sepal_len,0.627780,0.627780,0.213310
3,sepal_wid,0.570262,0.570262,0.193766
